In [34]:
import datetime as datetime
import pandas as pd
import math
import re
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pyodbc
import requests

short_start = "2019-3-01"
med_start = "2019-1-01"
long_start = "2018-01-01"
short_end = "2019-4-01"
med_end = "2019-4-01"
long_end = "2019-4-01"

drop_list = ['XRP' , 'XLM' , 'BCH' , 'LTC' , 'ADA' , 'TRX' , 'EOS' , 'BSV']
long_drop = ['currency' , 'platform' , 'application']

tfs = ['short' , 'med' , 'long']

df_all = dict.fromkeys(tfs)
df_benchmarks = dict.fromkeys(tfs)
returns_all = dict.fromkeys(tfs)
returns_benchmarks = dict.fromkeys(tfs)
results = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
net_return = dict.fromkeys(tfs)
vol = dict.fromkeys(tfs)
ann_factor = dict.fromkeys(tfs)
total_return = dict.fromkeys(tfs)
percent_sd = dict.fromkeys(tfs)
summary = dict.fromkeys(tfs)
corr = dict.fromkeys(tfs)
betas = dict.fromkeys(tfs)
beta_results = dict.fromkeys(tfs)
total_return_excel = dict.fromkeys(tfs)
sortino_df = dict.fromkeys(tfs)
sortino = dict.fromkeys(tfs)
average_return = dict.fromkeys(tfs)

data_bletchley = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\\bletchley_history.xlsx' , index_col= 0)
data_coins = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bletchley.xlsx' , index_col=0)
#data_bitcoin = pd.read_excel('C:/Users\stoin\Desktop\Mike\Jupyter\Data\coins_history_bitcoin.xlsx' , index_col=0)
data_bletchley['ERC20'] = data_bletchley['ethereum']
data_bletchley = data_bletchley.drop('ethereum' , axis=1)
data_bletch = data_bletchley

In [35]:
data_finance = data_bletch.fillna(method='ffill')
data_finance = data_finance.dropna()
index_data = data_finance
data_coins = data_coins.fillna(method='ffill')
data_coins = data_coins.dropna()

#bitcoin_data = data_bitcoin.fillna(method='ffill')
#bitcoin_data = bitcoin_data.dropna()
#bitcoin_data.index = (pd.to_datetime(bitcoin_data.index.values,unit='ms'))
#bitcoin_data.index = bitcoin_data.index.round('d')

index_data.index = (pd.to_datetime(index_data.index.values,unit='ms'))
index_data.index = index_data.index.round('d')
df = index_data.join(data_coins, how='inner')
#df = df.join(bitcoin_data,how='inner')
df

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2018-11-08,740.953740,637.900186,110.287135,311.303644,40.569745,27.086623,24.329500,73.888661,6495.77547,216.40519,...,5.61607,0.07786,6449.50503,9.69729,110.48793,1.31667,164.70680,0.50167,16.62824,9.60317
2018-11-09,721.565444,618.797851,106.479184,303.007814,39.834951,25.967035,23.624929,71.826111,6429.19559,210.59830,...,5.49626,0.07602,102.66811,9.53628,107.73183,1.30120,163.24740,0.48149,16.26065,9.42063
2018-11-10,713.346165,609.780474,103.749804,299.376551,39.246942,25.732236,23.221154,70.634206,6396.25716,209.48592,...,5.39397,0.07435,70.33582,9.54875,107.03976,1.28991,159.28152,0.47727,15.94697,9.50456
2018-11-11,716.586940,610.434094,104.335381,300.675133,39.365448,25.905354,23.253192,70.736600,6387.92027,211.99709,...,5.41039,0.07512,88.33960,9.54135,104.85884,1.30044,161.12468,0.48463,15.93261,9.46750
2018-11-12,719.176084,608.269451,102.901564,301.480361,39.486859,26.000422,22.972163,69.884891,6391.16890,210.78423,...,5.44913,0.07631,114.76690,9.43560,106.23488,1.30966,159.31131,0.48219,16.11287,9.22718
2018-11-13,716.724558,611.223194,101.636233,300.550535,39.208126,26.162344,22.709419,68.951585,6347.84405,209.76210,...,5.39862,0.07554,726.18475,9.31937,104.39629,1.25052,161.32332,0.49305,15.72473,9.13351
2018-11-14,708.465071,596.955295,98.718815,296.715644,38.893893,25.491212,22.114844,66.856327,6264.28369,201.34617,...,5.10806,0.06914,189.55126,8.31798,98.27927,1.16562,150.02181,0.48127,14.69142,8.64601
2018-11-15,639.335345,531.505568,86.586428,267.333560,35.057002,22.916454,19.560126,59.560033,5750.50513,182.04190,...,4.70168,0.06541,173.57708,8.29694,90.42572,1.09946,142.83311,0.43299,13.62862,7.96862
2018-11-16,628.338127,520.364846,84.732955,262.641013,34.171254,22.849624,19.211085,58.138167,5615.73645,180.43889,...,4.70969,0.06316,89.67520,8.07748,87.61479,1.10158,137.06012,0.41837,12.93027,7.68730
2018-11-17,612.375185,503.651023,82.562568,255.850709,33.395493,22.059975,18.680848,56.552413,5580.47433,174.63610,...,4.58339,0.06147,112.94693,7.88014,86.90498,1.10799,137.09438,0.41152,12.75134,7.47302


In [36]:
returns = df.pct_change()
returns = returns.dropna()

In [37]:
#filter master dataframes based on date ranges
returns_all['short'] = returns.loc[short_start:short_end]
returns_benchmarks['short'] = returns.loc[short_start:short_end]
df_all['short'] = df.loc[short_start:short_end]
df_benchmarks['short'] = df.loc[short_start:short_end]

returns_all['med'] = returns.loc[med_start:med_end]
returns_benchmarks['med'] = returns.loc[med_start:med_end]
df_all['med'] = df.loc[med_start:med_end]
df_benchmarks['med'] = df.loc[med_start:med_end]

returns_all['long'] = returns.loc[long_start:long_end]
returns_benchmarks['long'] = returns.loc[long_start:long_end]
df_all['long'] = df.loc[long_start:long_end]
df_benchmarks['long'] = df.loc[long_start:long_end]

#drop coins that aren't benchmarks
for c in drop_list:
    for t in tfs:
        returns_benchmarks[t] = returns_benchmarks[t].drop([c] , axis=1)
        df_benchmarks[t] = df_benchmarks[t].drop([c] , axis=1)

In [38]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-28,0.036965,0.046729,0.052198,0.038243,0.030804,0.068479,0.043738,0.041472,0.003272,0.023525,...,0.114836,0.060418,-0.002306,-0.001684,-0.003958,0.062808,0.030977,0.010056,0.020540,0.005198
2019-03-29,-0.007547,0.003870,0.017482,-0.005807,-0.005496,-0.006181,0.010195,0.009972,0.019501,0.004623,...,0.017344,0.000301,0.000767,0.028358,0.016670,0.108268,0.050395,0.017120,0.011669,0.004651
2019-03-30,0.022245,0.026551,0.030864,0.022888,0.019359,0.033630,0.022213,0.015133,0.001759,0.027042,...,-0.000901,0.066126,-0.009535,-0.029669,-0.005611,0.133295,-0.001603,-0.000549,0.064243,0.015950
2019-03-31,-0.002918,0.010540,-0.002028,-0.001791,-0.001604,-0.004097,0.003172,0.007400,-0.001418,-0.006815,...,-0.037652,0.012405,0.006731,0.027077,-0.002481,0.044340,0.025233,-0.008566,-0.002539,-0.019195
2019-04-01,-0.000982,0.037100,0.019033,0.002839,0.002085,0.005825,0.021809,0.022778,0.003046,0.000539,...,0.018173,-0.020468,0.008964,0.043674,0.056711,0.124530,0.107066,0.012291,0.027355,0.017529


In [39]:
#drop sectors that don't have data back to 1/1/2018
for i in long_drop:
    df_benchmarks['long'] = df_benchmarks['long'].drop([i] , axis=1)
    returns_benchmarks['long'] = returns_benchmarks['long'].drop([i] , axis=1)

In [40]:
#calculate sortino to btc
for t in tfs:
    sortino_df[t] = pd.DataFrame()
    sortino[t] = pd.DataFrame(index = ['sortino'])
    for col in returns_benchmarks[t]:
        sortino_df[t][col] = returns_benchmarks[t][col] - returns_benchmarks[t]['BTC']
        avg_ret = np.mean(sortino_df[t][col])
        a = [r ** 2 for r in sortino_df[t][col] if r < 0]
        a_sum = np.sum(a)
        b = len(sortino_df[t][col])
        down_risk = np.sqrt(a_sum/b)
        if down_risk > 0.000001:
            sortino[t][col] = (avg_ret / down_risk)*math.sqrt(365)
        else:
            sortino[t][col] = 0

In [41]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-28,0.036965,0.046729,0.052198,0.038243,0.030804,0.068479,0.043738,0.041472,0.003272,0.023525,...,0.114836,0.060418,-0.002306,-0.001684,-0.003958,0.062808,0.030977,0.010056,0.020540,0.005198
2019-03-29,-0.007547,0.003870,0.017482,-0.005807,-0.005496,-0.006181,0.010195,0.009972,0.019501,0.004623,...,0.017344,0.000301,0.000767,0.028358,0.016670,0.108268,0.050395,0.017120,0.011669,0.004651
2019-03-30,0.022245,0.026551,0.030864,0.022888,0.019359,0.033630,0.022213,0.015133,0.001759,0.027042,...,-0.000901,0.066126,-0.009535,-0.029669,-0.005611,0.133295,-0.001603,-0.000549,0.064243,0.015950
2019-03-31,-0.002918,0.010540,-0.002028,-0.001791,-0.001604,-0.004097,0.003172,0.007400,-0.001418,-0.006815,...,-0.037652,0.012405,0.006731,0.027077,-0.002481,0.044340,0.025233,-0.008566,-0.002539,-0.019195
2019-04-01,-0.000982,0.037100,0.019033,0.002839,0.002085,0.005825,0.021809,0.022778,0.003046,0.000539,...,0.018173,-0.020468,0.008964,0.043674,0.056711,0.124530,0.107066,0.012291,0.027355,0.017529


In [42]:
data_coins_week = data_coins.loc[short_start:short_end]
#data_coins_week = data_coins_week[:]
net_week_return = data_coins_week.pct_change(len(data_coins_week)-1)
ann_week_factor = 365.000/(len(data_coins_week))
data_coins_week_returns = pd.DataFrame()
data_coins_week_returns = data_coins_week.pct_change()
data_coins_week_returns = data_coins_week_returns.dropna()
total_week_return = net_week_return.tail(1)*100
data_coins_week = data_coins_week.append(total_week_return)
data_coins_week.index = data_coins_week.index[:-1].append(pd.Index(['return']))

In [43]:
data_coins_week

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-01 00:00:00,3805.974020,135.460610,0.313610,0.084460,45.658020,130.066380,0.023660,3.50252,0.042830,66.66180,10.288040,47.807540,0.40789,81.855410,0.28561,8.859190,4.22605
2019-03-02 00:00:00,3787.057410,135.315420,0.315310,0.084420,47.016150,131.135140,0.023390,3.44986,0.042780,66.41078,11.308180,48.249770,0.41554,82.248430,0.29541,8.923240,4.22759
2019-03-03 00:00:00,3785.145420,132.085610,0.311740,0.082820,48.204390,129.809070,0.022470,3.45026,0.042120,65.70003,11.671920,48.135530,0.39826,81.515580,0.29225,8.703680,4.21083
2019-03-04 00:00:00,3742.763300,128.733620,0.304920,0.085530,46.976700,127.791120,0.021930,3.45573,0.041150,63.52819,11.158350,47.982570,0.39171,78.517350,0.28183,8.491390,4.17741
2019-03-05 00:00:00,3862.783240,128.364410,0.310450,0.085130,47.015730,125.656780,0.023620,3.32557,0.041520,64.51409,11.877890,48.909360,0.41332,80.999180,0.28154,8.511490,4.22804
2019-03-06 00:00:00,3867.452970,136.830380,0.314360,0.085590,52.576890,131.891840,0.023700,3.74897,0.042790,66.47777,13.640950,49.210930,0.41522,82.552760,0.28237,8.734760,4.27528
2019-03-07 00:00:00,3854.152420,137.648940,0.316220,0.084530,55.488060,131.296900,0.023180,3.73318,0.042380,66.19706,14.554040,49.738440,0.41501,83.079710,0.28145,8.725970,4.27532
2019-03-08 00:00:00,3820.964590,136.383130,0.311080,0.084240,56.492980,129.661320,0.022710,3.72664,0.042270,65.88928,14.992070,49.568790,0.41362,81.354960,0.27626,9.114260,4.32900
2019-03-09 00:00:00,3911.350810,134.729770,0.310350,0.086970,56.177560,128.387950,0.022590,3.63154,0.042680,65.60422,14.489530,49.504500,0.41291,80.835820,0.27877,8.875230,4.24182
2019-03-10 00:00:00,3892.214700,135.978220,0.309920,0.089080,57.225280,132.064570,0.022470,3.71103,0.046060,65.56207,14.291160,49.124810,0.41244,82.204580,0.27988,8.863800,4.21655


In [44]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003730,0.003781,...,-0.000220,-0.002794,-0.009797,0.045107,-0.019217,0.006365,0.019256,-0.003906,-0.011235,-0.014845
2019-03-02,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004970,-0.001072,...,-0.015035,-0.001167,-0.003766,0.099158,0.009250,0.018755,0.004801,0.034313,0.007230,0.000364
2019-03-03,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000505,-0.023869,...,0.000116,-0.015428,-0.010702,0.032166,-0.002368,-0.041584,-0.008910,-0.010697,-0.024605,-0.003964
2019-03-04,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011197,-0.025377,...,0.001585,-0.023029,-0.033057,-0.044000,-0.003178,-0.016447,-0.036781,-0.035654,-0.024391,-0.007937
2019-03-05,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032067,-0.002868,...,-0.037665,0.008991,0.015519,0.064484,0.019315,0.055168,0.031609,-0.001029,0.002367,0.012120
2019-03-06,0.052274,0.084908,0.043091,0.054320,0.042553,0.079849,0.088721,0.095061,0.001209,0.065953,...,0.127317,0.030588,0.030438,0.148432,0.006166,0.004597,0.019180,0.002948,0.026232,0.011173
2019-03-07,0.002188,0.014242,-0.004632,0.002867,0.002098,0.000729,0.012888,0.015896,-0.003439,0.005982,...,-0.004212,-0.009582,-0.004223,0.066937,0.010719,-0.000506,0.006383,-0.003258,-0.001006,0.000009
2019-03-08,0.002430,0.023899,0.030511,0.004814,0.004598,0.006727,0.027980,0.021305,-0.008611,-0.009196,...,-0.001752,-0.002596,-0.004649,0.030097,-0.003411,-0.003349,-0.020760,-0.018440,0.044498,0.012556
2019-03-09,-0.011954,-0.031500,0.002758,-0.012987,-0.007684,-0.024343,-0.023665,-0.028403,0.023655,-0.012123,...,-0.025519,0.009700,-0.004326,-0.033520,-0.001297,-0.001717,-0.006381,0.009086,-0.026226,-0.020139
2019-03-10,0.021451,0.028492,0.048644,0.022784,0.019861,0.033830,0.034807,0.031074,-0.004892,0.009266,...,0.021889,0.079194,-0.000642,-0.013691,-0.007670,-0.001138,0.016933,0.003982,-0.001288,-0.005957


In [45]:
print ('short start: ', returns_benchmarks['short'].index[0])
print ('medium start: ', returns_benchmarks['med'].index[0])
print ('long start: ', returns_benchmarks['long'].index[0])
print ('end: ', returns_benchmarks['short'].index[-2])
print ('end: ', returns_benchmarks['med'].index[-2])
print ('end: ', returns_benchmarks['long'].index[-2])

short start:  2019-03-01 00:00:00
medium start:  2019-01-01 00:00:00
long start:  2018-11-09 00:00:00
end:  2019-03-31 00:00:00
end:  2019-03-31 00:00:00
end:  2019-03-31 00:00:00


In [46]:
returns_benchmarks['short'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-01,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003730,0.003781,...,-0.000220,-0.002794,-0.009797,0.045107,-0.019217,0.006365,0.019256,-0.003906,-0.011235,-0.014845
2019-03-02,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004970,-0.001072,...,-0.015035,-0.001167,-0.003766,0.099158,0.009250,0.018755,0.004801,0.034313,0.007230,0.000364
2019-03-03,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000505,-0.023869,...,0.000116,-0.015428,-0.010702,0.032166,-0.002368,-0.041584,-0.008910,-0.010697,-0.024605,-0.003964
2019-03-04,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011197,-0.025377,...,0.001585,-0.023029,-0.033057,-0.044000,-0.003178,-0.016447,-0.036781,-0.035654,-0.024391,-0.007937
2019-03-05,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032067,-0.002868,...,-0.037665,0.008991,0.015519,0.064484,0.019315,0.055168,0.031609,-0.001029,0.002367,0.012120


In [47]:
print (data_coins_week.head(n=1))
data_coins_week.tail(n=1)

                            BTC        ETH      XRP      XLM       LTC  \
2019-03-01 00:00:00  3805.97402  135.46061  0.31361  0.08446  45.65802   

                           BCH      TRX      EOS      ADA      BSV       BNB  \
2019-03-01 00:00:00  130.06638  0.02366  3.50252  0.04283  66.6618  10.28804   

                          XMR      XTZ      DASH      IOT      NEO      ETC  
2019-03-01 00:00:00  47.80754  0.40789  81.85541  0.28561  8.85919  4.22605  


,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
return,7.799758,5.084061,-0.911961,28.688136,33.599574,30.421082,-0.380389,20.40331,64.254028,-2.05425,71.706759,18.049559,159.08946,35.873292,8.71118,15.031623,14.99438


In [48]:
annualize = math.sqrt(365)
for t in tfs:
    results[t] = returns_benchmarks[t].describe()
    vol[t] = pd.DataFrame()
    vol[t] = results[t].loc['std']*annualize*100
    vol[t].name = 'annualized volatility'
    results[t] = results[t].append(df_benchmarks[t].iloc[0])
    results[t].index = results[t].index[:-1].append(pd.Index(['open price']))
    results[t] = results[t].append(df_benchmarks[t][-1:])
    results[t].index = results[t].index[:-1].append(pd.Index(['last price']))
    results[t] = results[t].append(vol[t])

In [49]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-28,0.036965,0.046729,0.052198,0.038243,0.030804,0.068479,0.043738,0.041472,0.003272,0.023525,...,0.114836,0.060418,-0.002306,-0.001684,-0.003958,0.062808,0.030977,0.010056,0.020540,0.005198
2019-03-29,-0.007547,0.003870,0.017482,-0.005807,-0.005496,-0.006181,0.010195,0.009972,0.019501,0.004623,...,0.017344,0.000301,0.000767,0.028358,0.016670,0.108268,0.050395,0.017120,0.011669,0.004651
2019-03-30,0.022245,0.026551,0.030864,0.022888,0.019359,0.033630,0.022213,0.015133,0.001759,0.027042,...,-0.000901,0.066126,-0.009535,-0.029669,-0.005611,0.133295,-0.001603,-0.000549,0.064243,0.015950
2019-03-31,-0.002918,0.010540,-0.002028,-0.001791,-0.001604,-0.004097,0.003172,0.007400,-0.001418,-0.006815,...,-0.037652,0.012405,0.006731,0.027077,-0.002481,0.044340,0.025233,-0.008566,-0.002539,-0.019195
2019-04-01,-0.000982,0.037100,0.019033,0.002839,0.002085,0.005825,0.021809,0.022778,0.003046,0.000539,...,0.018173,-0.020468,0.008964,0.043674,0.056711,0.124530,0.107066,0.012291,0.027355,0.017529


In [50]:
for t in tfs:
    average_return[t] = pd.DataFrame()
    average_return[t] = returns_benchmarks[t]
    average_return[t].loc['mean'] = returns_benchmarks[t].mean(axis=0)
    returns_benchmarks[t] = returns_benchmarks[t][:-1]

C:\Users\stoin\Anaconda2\envs\py35\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [51]:
returns_benchmarks['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-01 00:00:00,0.000089,0.001125,0.002772,0.000236,-0.001178,0.000330,0.014558,0.005148,-0.003730,0.003781,...,-0.000220,-0.002794,-0.009797,0.045107,-0.019217,0.006365,0.019256,-0.003906,-0.011235,-0.014845
2019-03-02 00:00:00,0.001945,0.033469,0.021862,0.004711,0.003317,0.005152,0.032925,0.036816,-0.004970,-0.001072,...,-0.015035,-0.001167,-0.003766,0.099158,0.009250,0.018755,0.004801,0.034313,0.007230,0.000364
2019-03-03 00:00:00,-0.002430,-0.000707,-0.000303,-0.002244,0.001227,-0.009133,0.001460,0.000615,-0.000505,-0.023869,...,0.000116,-0.015428,-0.010702,0.032166,-0.002368,-0.041584,-0.008910,-0.010697,-0.024605,-0.003964
2019-03-04 00:00:00,-0.011021,-0.021609,-0.008489,-0.011707,-0.009177,-0.013162,-0.019840,-0.021733,-0.011197,-0.025377,...,0.001585,-0.023029,-0.033057,-0.044000,-0.003178,-0.016447,-0.036781,-0.035654,-0.024391,-0.007937
2019-03-05 00:00:00,-0.024378,-0.029798,-0.037056,-0.025141,-0.021542,-0.042697,-0.015249,-0.012223,0.032067,-0.002868,...,-0.037665,0.008991,0.015519,0.064484,0.019315,0.055168,0.031609,-0.001029,0.002367,0.012120
2019-03-06 00:00:00,0.052274,0.084908,0.043091,0.054320,0.042553,0.079849,0.088721,0.095061,0.001209,0.065953,...,0.127317,0.030588,0.030438,0.148432,0.006166,0.004597,0.019180,0.002948,0.026232,0.011173
2019-03-07 00:00:00,0.002188,0.014242,-0.004632,0.002867,0.002098,0.000729,0.012888,0.015896,-0.003439,0.005982,...,-0.004212,-0.009582,-0.004223,0.066937,0.010719,-0.000506,0.006383,-0.003258,-0.001006,0.000009
2019-03-08 00:00:00,0.002430,0.023899,0.030511,0.004814,0.004598,0.006727,0.027980,0.021305,-0.008611,-0.009196,...,-0.001752,-0.002596,-0.004649,0.030097,-0.003411,-0.003349,-0.020760,-0.018440,0.044498,0.012556
2019-03-09 00:00:00,-0.011954,-0.031500,0.002758,-0.012987,-0.007684,-0.024343,-0.023665,-0.028403,0.023655,-0.012123,...,-0.025519,0.009700,-0.004326,-0.033520,-0.001297,-0.001717,-0.006381,0.009086,-0.026226,-0.020139
2019-03-10 00:00:00,0.021451,0.028492,0.048644,0.022784,0.019861,0.033830,0.034807,0.031074,-0.004892,0.009266,...,0.021889,0.079194,-0.000642,-0.013691,-0.007670,-0.001138,0.016933,0.003982,-0.001288,-0.005957


In [52]:
for t in tfs:
    net_return[t] = df_benchmarks[t].pct_change(len(df_benchmarks[t])-1)
    ann_factor[t] = 365.000/(len(df_benchmarks[t]))
    total_return[t] = net_return[t].tail(1)*100
    total_return_excel[t] = net_return[t].tail(1)
    percent_sd[t] = results[t].iloc[2,:]*100
    results[t] = results[t].append(percent_sd[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['daily volatility']))
    #print total_return_excel[t]
    results[t] = results[t].append(total_return_excel[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['return']))
    #print results[t]
    #print average_return[t]/(percent_sd[t]*0.01).transpose()
    Sharpe_values = (average_return[t].loc['mean']/(percent_sd[t]*0.01))*annualize
    Sharpe_values = Sharpe_values.rename('sharpe_values')
    results[t] = results[t].append(Sharpe_values)
    results[t].index = results[t].index[:-1].append(pd.Index(['Sharpe']))
    results[t] = results[t].append(sortino[t])
    results[t].index = results[t].index[:-1].append(pd.Index(['Sortino']))
#print (results[t])

In [53]:
returns_benchmarks['med'].head()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-01-01 00:00:00,-0.046381,-0.050800,-0.056148,-0.046939,-0.042739,-0.057864,-0.040122,-0.046808,-0.015306,-0.035913,...,-0.018984,-0.009976,-0.000699,0.104085,0.004264,-0.034884,0.025143,0.019577,-0.026870,-0.010181
2019-01-02 00:00:00,0.046762,0.043521,0.042677,0.046447,0.043137,0.051576,0.036109,0.039667,0.046285,0.057667,...,0.035418,0.036948,0.080048,-0.014789,0.043582,0.037096,0.021561,0.035972,0.047718,0.055428
2019-01-03 00:00:00,0.033371,0.035948,0.027412,0.033396,0.022256,0.053596,0.031989,0.027348,-0.016791,0.082061,...,0.057606,0.016890,-0.004952,-0.048996,0.038191,-0.040803,-0.004683,0.038350,-0.026585,-0.018396
2019-01-04 00:00:00,-0.041354,-0.043182,-0.034871,-0.041317,-0.036859,-0.054475,-0.028133,-0.029247,-0.003201,-0.027497,...,-0.047362,-0.025711,-0.046443,-0.003529,-0.021608,0.012005,-0.018672,-0.016232,-0.017978,-0.035560
2019-01-05 00:00:00,0.018170,0.025959,0.024532,0.018870,0.013260,0.025122,0.048155,0.050412,0.003058,0.039930,...,0.009743,0.020318,-0.010235,0.017679,0.009565,0.006614,-0.004844,-0.013472,-0.014271,0.031398


In [54]:
for t in tfs:
    summary[t] = results[t].iloc[8:]
    corr[t] = returns_benchmarks[t].corr(method='spearman')
    betas[t] = returns_benchmarks[t].cov()
    beta_results[t] = pd.DataFrame()
    beta_results[t] = betas[t].iloc[0,:] / betas[t].iloc[3,0]
    corr[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_corr'+t+'.csv')
    summary[t].to_csv('C:/Users\stoin\Desktop\Mike\Jupyter\Output\\bletch_output_results'+t+'.csv')
beta_all = pd.DataFrame()
beta_all['MTD'] = beta_results['short']
beta_all['QTD'] = beta_results['med']
beta_all['YTD'] = beta_results['long']

In [55]:
returns_benchmarks['short'].tail()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
2019-03-28 00:00:00,0.036965,0.046729,0.052198,0.038243,0.030804,0.068479,0.043738,0.041472,0.003272,0.023525,...,0.114836,0.060418,-0.002306,-0.001684,-0.003958,0.062808,0.030977,0.010056,0.020540,0.005198
2019-03-29 00:00:00,-0.007547,0.003870,0.017482,-0.005807,-0.005496,-0.006181,0.010195,0.009972,0.019501,0.004623,...,0.017344,0.000301,0.000767,0.028358,0.016670,0.108268,0.050395,0.017120,0.011669,0.004651
2019-03-30 00:00:00,0.022245,0.026551,0.030864,0.022888,0.019359,0.033630,0.022213,0.015133,0.001759,0.027042,...,-0.000901,0.066126,-0.009535,-0.029669,-0.005611,0.133295,-0.001603,-0.000549,0.064243,0.015950
2019-03-31 00:00:00,-0.002918,0.010540,-0.002028,-0.001791,-0.001604,-0.004097,0.003172,0.007400,-0.001418,-0.006815,...,-0.037652,0.012405,0.006731,0.027077,-0.002481,0.044340,0.025233,-0.008566,-0.002539,-0.019195
2019-04-01 00:00:00,-0.000982,0.037100,0.019033,0.002839,0.002085,0.005825,0.021809,0.022778,0.003046,0.000539,...,0.018173,-0.020468,0.008964,0.043674,0.056711,0.124530,0.107066,0.012291,0.027355,0.017529


In [56]:
summary['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
open price,424.914158,381.398019,57.301770,178.648490,22.516762,15.764553,17.515368,50.875261,3805.974020,135.460610,...,3.502520,0.042830,66.661800,10.288040,47.807540,0.407890,81.855410,0.285610,8.859190,4.226050
last price,461.165079,519.768219,75.888100,198.575716,24.687741,18.616223,23.289193,68.247304,4102.830790,142.347510,...,4.217150,0.070350,65.292400,17.665260,56.436590,1.056800,111.219640,0.310490,10.190870,4.859720
annualized volatility,30.811426,49.210901,43.184668,31.687041,26.498053,47.262280,48.620221,51.298720,19.544811,35.414366,...,67.430307,73.650775,27.628475,88.035740,33.373642,150.747774,56.301977,41.072334,43.875291,31.493877
daily volatility,1.612744,2.575816,2.260389,1.658576,1.386971,2.473821,2.544899,2.685098,1.023022,1.853673,...,3.529464,3.855058,1.446140,4.608001,1.746856,7.890499,2.946980,2.149824,2.296538,1.648465
return,0.085314,0.362797,0.324359,0.111544,0.096416,0.180891,0.329643,0.341463,0.077998,0.050841,...,0.204033,0.642540,-0.020542,0.717068,0.180496,1.590895,0.358733,0.087112,0.150316,0.149944
Sharpe,3.185025,7.469405,7.734205,3.972744,4.043965,4.254980,7.286890,6.919929,4.263350,1.888643,...,3.456418,8.046918,-1.129258,8.053779,5.178690,8.019961,6.892291,2.409393,3.563701,4.678185
Sortino,0.550507,8.617705,8.414259,1.567523,0.955608,3.461514,8.929146,8.806559,0.000000,-0.718250,...,4.321520,21.066262,-5.472537,18.397565,6.229531,24.241206,17.243761,0.719801,2.724076,3.093053


In [66]:
pd.set_option('display.max_columns', 500)
summary['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
open price,420.511599,337.862280,54.644247,175.290083,22.237102,15.347474,13.168780,39.924454,3763.079330,133.640420,0.358190,0.113610,30.753500,151.340060,0.019310,2.581740,0.041680,86.177510,6.354030,47.139050,0.473100,80.824970,0.361950,7.697350,5.107160
last price,461.165079,519.768219,75.888100,198.575716,24.687741,18.616223,23.289193,68.247304,4102.830790,142.347510,0.310750,0.108690,60.998920,169.633980,0.023570,4.217150,0.070350,65.292400,17.665260,56.436590,1.056800,111.219640,0.310490,10.190870,4.859720
annualized volatility,56.183788,65.944891,60.786214,56.428826,50.788681,72.914821,64.735798,67.242648,33.107617,69.486567,44.307517,61.399584,83.270801,75.799898,78.069240,81.418600,66.817023,63.159541,91.016553,51.555354,110.879903,59.343845,56.258518,59.929922,54.637101
daily volatility,2.940794,3.451713,3.181696,2.953620,2.658401,3.816536,3.388427,3.519641,1.732932,3.637093,2.319161,3.213801,4.358593,3.967548,4.086331,4.261644,3.497363,3.305921,4.764024,2.698531,5.803719,3.106199,2.944705,3.136875,2.859837
return,0.096676,0.538403,0.388767,0.132841,0.110205,0.212983,0.768516,0.709411,0.090285,0.065153,-0.132444,-0.043306,0.983479,0.120880,0.220611,0.633453,0.687860,-0.242350,1.780166,0.197236,1.233777,0.376055,-0.142174,0.323945,-0.048450
Sharpe,0.603549,2.661515,2.113944,0.832638,0.738891,1.107313,3.645724,3.283891,1.034094,0.499619,-1.255442,-0.006803,3.704832,0.670702,1.406634,2.748035,3.452516,-1.474412,5.475738,1.706743,3.329655,2.644682,-0.680906,2.009815,-0.171137
Sortino,-0.009897,3.685127,2.679485,0.384782,0.111801,1.075492,5.704734,5.115060,0.000000,0.012533,-3.856756,-1.066480,8.335511,0.406712,1.699901,5.079176,7.685289,-3.212975,9.660527,2.200432,6.936231,4.693520,-2.134143,2.966084,-1.230898


In [58]:
summary['long']

,ten,twenty,forty,total,ERC20,BTC,ETH,XRP,XLM,LTC,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
open price,740.953740,637.900186,110.287135,311.303644,73.888661,6495.775470,216.405190,0.533980,0.254740,53.739200,...,5.616070,0.077860,6449.505030,9.697290,110.487930,1.316670,164.706800,0.501670,16.628240,9.603170
last price,461.165079,519.768219,75.888100,198.575716,68.247304,4102.830790,142.347510,0.310750,0.108690,60.998920,...,4.217150,0.070350,65.292400,17.665260,56.436590,1.056800,111.219640,0.310490,10.190870,4.859720
annualized volatility,77.020808,89.148486,90.618905,77.803867,91.689223,58.372141,87.923921,57.084036,73.183314,89.212608,...,100.347704,80.073644,897.493803,102.303028,72.861036,116.775761,86.401288,79.077720,77.603092,76.696679
daily volatility,4.031453,4.666245,4.743210,4.072441,4.799233,3.055338,4.602148,2.987915,3.830590,4.669601,...,5.252439,4.191246,46.976973,5.354785,3.813721,6.112323,4.522450,4.139117,4.061932,4.014488
return,-0.377606,-0.185189,-0.311904,-0.362116,-0.076349,-0.368385,-0.342218,-0.418049,-0.573330,0.135092,...,-0.249092,-0.096455,-0.989876,0.821670,-0.489206,-0.197369,-0.324742,-0.381087,-0.387135,-0.493946
Sharpe,-1.182654,-0.136273,-0.592293,-1.082222,0.240424,-1.711809,-0.778371,-2.136131,-2.595695,0.794086,...,-0.241308,0.075870,0.983017,2.012084,-1.982133,0.084003,-0.728557,-1.154556,-1.218395,-1.874858
Sortino,0.290256,2.320094,1.206395,0.511433,3.209580,0.000000,0.805580,-0.606205,-1.949933,5.084674,...,1.549358,2.816007,4.267082,4.333279,-1.053298,1.562559,0.742426,0.259518,0.143990,-1.068770


In [59]:
corr['short']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
ten,1.000000,0.776026,0.789956,0.987537,0.981672,0.973974,0.735337,0.711877,0.050220,0.782258,...,0.539223,0.578446,0.251100,0.120968,0.183284,0.083578,0.197947,0.182918,0.495601,0.398827
twenty,0.776026,1.000000,0.789589,0.843475,0.815249,0.800220,0.894428,0.901393,-0.147727,0.527493,...,0.357038,0.168622,0.181818,0.421188,0.129399,0.241202,0.237537,0.170088,0.369135,0.321481
forty,0.789956,0.789589,1.000000,0.820381,0.803886,0.840543,0.806452,0.770161,-0.016129,0.570381,...,0.468842,0.410191,0.290689,0.083944,0.125733,0.057551,0.258798,0.334311,0.539589,0.354839
total,0.987537,0.843475,0.820381,1.000000,0.984604,0.975806,0.783358,0.768695,0.002566,0.751100,...,0.505865,0.523827,0.262830,0.181085,0.182918,0.118402,0.223974,0.177419,0.500733,0.387463
currency,0.981672,0.815249,0.803886,0.984604,1.000000,0.954545,0.733138,0.718475,0.063050,0.741935,...,0.524194,0.520894,0.232771,0.143695,0.250367,0.122801,0.172654,0.229839,0.495601,0.408724
platform,0.973974,0.800220,0.840543,0.975806,0.954545,1.000000,0.736437,0.715909,0.001466,0.784091,...,0.575880,0.559018,0.339076,0.074413,0.157625,0.159091,0.256232,0.242302,0.540323,0.446481
application,0.735337,0.894428,0.806452,0.783358,0.733138,0.736437,1.000000,0.979839,-0.197581,0.520161,...,0.353006,0.222874,0.226906,0.516862,-0.016862,-0.041789,0.280425,0.037757,0.354106,0.194282
ERC20,0.711877,0.901393,0.770161,0.768695,0.718475,0.715909,0.979839,1.000000,-0.164956,0.521994,...,0.337243,0.215909,0.275293,0.587977,0.079179,-0.004032,0.316716,0.066349,0.370968,0.202713
BTC,0.050220,-0.147727,-0.016129,0.002566,0.063050,0.001466,-0.197581,-0.164956,1.000000,0.380865,...,0.045455,0.410557,0.353739,-0.008065,0.656158,0.167155,0.409824,0.417155,0.361437,0.284457
ETH,0.782258,0.527493,0.570381,0.751100,0.741935,0.784091,0.520161,0.521994,0.380865,1.000000,...,0.638563,0.666789,0.539589,0.197581,0.384531,0.240836,0.483138,0.389296,0.628299,0.468109


In [60]:
corr['med']

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
ten,1.000000,0.863588,0.860740,0.995967,0.992444,0.974433,0.797835,0.783232,0.279681,0.793176,...,0.617467,0.640318,0.345170,-0.105239,0.301955,-0.037906,0.143598,0.524633,0.490011,0.419237
twenty,0.863588,1.000000,0.821410,0.894209,0.865959,0.900202,0.923842,0.922822,0.251447,0.673754,...,0.589447,0.541787,0.244812,0.133621,0.348841,0.099642,0.195325,0.493682,0.538165,0.406182
forty,0.860740,0.821410,1.000000,0.878536,0.859423,0.889386,0.804898,0.774952,0.297691,0.709166,...,0.596773,0.617862,0.423567,-0.112450,0.317315,-0.026234,0.224777,0.540454,0.534280,0.461794
total,0.995967,0.894209,0.878536,1.000000,0.989365,0.979866,0.832243,0.818068,0.271433,0.791019,...,0.616940,0.636729,0.352793,-0.074734,0.304737,-0.016093,0.152603,0.530444,0.505947,0.424851
currency,0.992444,0.865959,0.859423,0.989365,1.000000,0.956159,0.797703,0.782426,0.295304,0.770638,...,0.599951,0.609450,0.333811,-0.095543,0.299963,-0.041528,0.129193,0.525867,0.475162,0.418068
platform,0.974433,0.900202,0.889386,0.979866,0.956159,1.000000,0.816735,0.801424,0.260123,0.807203,...,0.661999,0.672042,0.364382,-0.092876,0.326863,0.029016,0.184640,0.562267,0.550694,0.445742
application,0.797835,0.923842,0.804898,0.832243,0.797703,0.816735,1.000000,0.985858,0.269128,0.632811,...,0.573511,0.560094,0.255579,0.251957,0.290233,-0.008240,0.188344,0.414232,0.508878,0.374952
ERC20,0.783232,0.922822,0.774952,0.818068,0.782426,0.801424,0.985858,1.000000,0.261506,0.626258,...,0.552323,0.558464,0.267597,0.257340,0.320904,0.011565,0.199177,0.400222,0.505832,0.372219
BTC,0.279681,0.251447,0.297691,0.271433,0.295304,0.260123,0.269128,0.261506,1.000000,0.378722,...,0.449924,0.534461,0.319142,0.168852,0.585974,0.288768,0.492727,0.439733,0.437725,0.517833
ETH,0.793176,0.673754,0.709166,0.791019,0.770638,0.807203,0.632811,0.626258,0.378722,1.000000,...,0.696835,0.705676,0.582352,0.090373,0.465415,0.201037,0.397621,0.642902,0.600675,0.527810


In [61]:
corr['long']

,ten,twenty,forty,total,ERC20,BTC,ETH,XRP,XLM,LTC,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
ten,1.000000,0.904609,0.914549,0.997854,0.853725,0.618138,0.816294,0.500550,0.389442,0.669888,...,0.652841,0.613255,0.236527,-0.068030,0.391617,0.044769,0.209474,0.639372,0.578245,0.540210
twenty,0.904609,1.000000,0.899188,0.924004,0.944113,0.569245,0.772346,0.467723,0.350676,0.671653,...,0.633298,0.598198,0.202186,0.091003,0.451935,0.149139,0.273039,0.648975,0.631607,0.581302
forty,0.914549,0.899188,1.000000,0.927686,0.867400,0.628542,0.793575,0.500607,0.407322,0.671329,...,0.639405,0.634739,0.253735,-0.005992,0.437666,0.099654,0.317107,0.664959,0.624869,0.596256
total,0.997854,0.924004,0.927686,1.000000,0.873539,0.612906,0.818609,0.499109,0.386593,0.677751,...,0.651975,0.614285,0.236991,-0.053170,0.396410,0.060376,0.219069,0.643303,0.589009,0.549948
ERC20,0.853725,0.944113,0.867400,0.873539,1.000000,0.583161,0.717973,0.467050,0.384611,0.669018,...,0.601169,0.631361,0.196547,0.203511,0.434777,0.127434,0.265065,0.580965,0.609479,0.557856
BTC,0.618138,0.569245,0.628542,0.612906,0.583161,1.000000,0.559330,0.578799,0.529884,0.630113,...,0.567431,0.587076,0.256037,0.168632,0.535958,0.237409,0.434588,0.629223,0.555476,0.568133
ETH,0.816294,0.772346,0.793575,0.818609,0.717973,0.559330,1.000000,0.693900,0.531198,0.747800,...,0.708087,0.724519,0.395310,0.114523,0.593199,0.287157,0.449888,0.724597,0.701738,0.661713
XRP,0.500550,0.467723,0.500607,0.499109,0.467050,0.578799,0.693900,1.000000,0.744320,0.640669,...,0.573008,0.720854,0.419576,0.308858,0.684892,0.423163,0.534210,0.580498,0.612623,0.603360
XLM,0.389442,0.350676,0.407322,0.386593,0.384611,0.529884,0.531198,0.744320,1.000000,0.544921,...,0.499150,0.640496,0.369546,0.318543,0.627512,0.418550,0.509386,0.512156,0.554028,0.531633
LTC,0.669888,0.671653,0.671329,0.677751,0.669018,0.630113,0.747800,0.640669,0.544921,1.000000,...,0.754765,0.717887,0.369846,0.326106,0.606130,0.309662,0.512968,0.705026,0.677833,0.633536


In [62]:
beta_all

,MTD,QTD,YTD
ten,0.975741,0.996686,0.990408
twenty,1.279989,1.052475,1.089898
forty,1.048482,0.960198,1.102636
total,1.000000,1.000000,1.000000
currency,0.832804,0.895265,NaN
platform,1.450904,1.268748,NaN
application,1.242308,0.941919,NaN
ERC20,1.267729,0.953299,1.062998
BTC,-0.087445,0.296628,0.609474
ETH,0.930037,1.070957,0.980684


In [63]:
coins_high_low = data_coins_week_returns.describe()
coins_high_low

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
count,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000
mean,0.002477,0.001770,-0.000239,0.008656,0.009789,0.009067,0.000220,0.006598,0.016851,-0.000566,0.018597,0.005508,0.033986,0.010353,0.002925,0.004784,0.004646
std,0.010339,0.018840,0.010837,0.032152,0.029342,0.031376,0.026843,0.035857,0.039028,0.014607,0.046599,0.017192,0.080056,0.029914,0.021819,0.023167,0.016387
min,-0.011197,-0.025377,-0.021877,-0.041952,-0.027196,-0.030890,-0.051474,-0.037665,-0.038749,-0.033057,-0.044000,-0.024481,-0.094627,-0.036781,-0.035654,-0.033514,-0.020139
25%,-0.004931,-0.009786,-0.005909,-0.012722,-0.010582,-0.013776,-0.016220,-0.014827,-0.010894,-0.009137,-0.012969,-0.004785,-0.005151,-0.007646,-0.009631,-0.008154,-0.006834
50%,0.000165,-0.001072,-0.000966,0.005404,0.006524,-0.004330,-0.004374,-0.000842,0.008991,-0.003766,0.004281,0.000432,0.009631,0.002858,-0.000097,0.006849,0.002169
75%,0.005716,0.008225,0.005661,0.025854,0.020806,0.024412,0.008780,0.015907,0.047727,0.005858,0.040052,0.013520,0.065075,0.018261,0.011174,0.015867,0.012876
max,0.032067,0.065953,0.020830,0.108442,0.118283,0.086975,0.077063,0.127317,0.103343,0.030438,0.148432,0.056711,0.316711,0.107066,0.070883,0.064243,0.045234


In [64]:
returns_benchmarks['short'].describe()

,ten,twenty,forty,total,currency,platform,application,ERC20,BTC,ETH,...,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
count,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,...,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000,32.000000
mean,0.002689,0.010071,0.009151,0.003449,0.002936,0.005510,0.009707,0.009726,0.002283,0.001832,...,0.006385,0.016237,-0.000855,0.019425,0.004735,0.033123,0.010631,0.002711,0.004284,0.004037
std,0.016127,0.025758,0.022604,0.016586,0.013870,0.024738,0.025449,0.026851,0.010230,0.018537,...,0.035295,0.038551,0.014461,0.046080,0.017469,0.078905,0.029470,0.021498,0.022965,0.016485
min,-0.024378,-0.031500,-0.037056,-0.025141,-0.021542,-0.042697,-0.032617,-0.032548,-0.011197,-0.025377,...,-0.037665,-0.038749,-0.033057,-0.044000,-0.024481,-0.094627,-0.036781,-0.035654,-0.033514,-0.020139
25%,-0.006765,-0.007606,-0.002679,-0.004270,-0.004434,-0.010729,-0.011272,-0.011806,-0.004912,-0.009491,...,-0.014723,-0.010458,-0.009378,-0.012609,-0.005612,-0.004356,-0.007013,-0.009099,-0.009123,-0.007767
50%,0.000551,0.009654,0.006721,0.002170,0.001648,0.002940,0.010695,0.006683,-0.000170,-0.000266,...,-0.000531,0.006478,-0.003994,0.010207,0.000004,0.008400,0.003830,-0.000323,0.004608,0.001267
75%,0.005534,0.024562,0.022866,0.008185,0.007006,0.015595,0.027511,0.024852,0.005056,0.007960,...,0.015188,0.044643,0.005421,0.044032,0.012791,0.063942,0.019199,0.010615,0.015339,0.012716
max,0.052274,0.084908,0.052198,0.054320,0.042553,0.079849,0.088721,0.095061,0.032067,0.065953,...,0.127317,0.103343,0.030438,0.148432,0.056711,0.316711,0.107066,0.070883,0.064243,0.045234


In [65]:
data_coins_week.tail(n=1)

,BTC,ETH,XRP,XLM,LTC,BCH,TRX,EOS,ADA,BSV,BNB,XMR,XTZ,DASH,IOT,NEO,ETC
return,7.799758,5.084061,-0.911961,28.688136,33.599574,30.421082,-0.380389,20.40331,64.254028,-2.05425,71.706759,18.049559,159.08946,35.873292,8.71118,15.031623,14.99438
